
<div align="center">

<h1><font color='#fd7317'>Generación del modelo 182 a partir de los datos del fichero CSV de CALM</font></h1>
<h3><font color='#999999'>(para la FPV)</font></h3>
<img  width="100" src="https://www.hacienda.gob.es/es-ES/Prensa/En%20Portada/2020/PublishingImages/20181307_AEAT.jpg" />
</div>

### imports

In [61]:
#Code tells you how; Comments tell you why."
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

### Definicion de Funciones propias ```validate_DNI_NIE()``` ```extract_DNI_dana()```

Ojo! Verificar que los ficheros pasados a ```extract_DNI_dana()``` no tienen malformaciones: 
<img src="img\error1.png" width="800">

In [112]:
def validate_DNI_NIE(dni):
    '''
   Valida el documento, tanto extranjero como nacional

   :param: str filename: file name of the exported mod182 file 
   :return: True si es un fichero válido, False en caso contrario
   '''
    tabla = "TRWAGMYFPDXBNJZSQVHLCKE"
    dig_ext = "XYZ"
    reemp_dig_ext = {'X':'0', 'Y':'1', 'Z':'2'}
    numeros = "1234567890"
    dni = str(dni)
    dni = dni.upper()
    if len(dni) == 9:
        dig_control = dni[8]
        dni = dni[:8]
        if dni[0] in dig_ext:
            dni = dni.replace(dni[0], reemp_dig_ext[dni[0]])
        return len(dni) == len([n for n in dni if n in numeros]) \
            and tabla[int(dni)%23] == dig_control
    return False


def extract_DNI_dana(filename):
    '''
   Extracts DNI and dana from an 'oficial' modelo182 file

   :param str filename: file name of the exported mod182 file 
   :return: dataframe with columns ["dni", "dana"]
   '''
    dfprev = pd.read_csv(r'.\modelo182\\' + filename, skiprows=1, header=None)
    dni = dfprev[0].str[17:26] # dni
    dana = (dfprev.iloc[:, 0].str[84:96]).astype(float) / 100 # donation
    return pd.concat([dni, dana], axis=1)
    

### Constantes y carga del CSV 

In [117]:
# CONSTANTES anuales (actualizar cada año)
EJERCICIO = "2019"
NIF_DECLARANTE = "G17572108"
DENOMINACION_DECLARANTE = "FUNDACION PRIVADA VIPASSANA" 
PERSONA_CONTACTO = "KARMASS MOHAMED"
TELEFONO_CONTACTO = "972426090"
NUMERO_JUSTIFICANTE_DECLARACION = "1822628017603"
FICHERO_EXPORTACION_ANIO_ANTERIOR   = "exportacion_2018.txt"
FICHERO_EXPORTACION_ANIO_ANTERIOR_2 = "exportacion_2017.txt"

# CONSTANTES
TIPO_REGISTRO = "1" 
MODELO_DECLARACION = "182"
TIPO_SOPORTE = "T" 

# Load files
# df = pd.read_csv(r".\modelo182\Calm4AdHocReport_279_1579686916.csv")
df = pd.read_csv(r".\modelo182\calm4_dana_2019.csv")
df1 = extract_DNI_dana(FICHERO_EXPORTACION_ANIO_ANTERIOR)
df2 = extract_DNI_dana(FICHERO_EXPORTACION_ANIO_ANTERIOR_2)

### REGISTRO TIPO 1 (pdf Hacienda, pág.2)

### REGISTROS TIPO 2 (pdf Hacienda, pág.9)

In [64]:
# REGISTROS TIPO 2 (pdf Hacienda, pág.9)
row_beginning = "2" + MODELO_DECLARACION + EJERCICIO + NIF_DECLARANTE
count = len(df["National Id"])

#Creacion e inicialización del DataFrame resultado
#TIPO_DE_REGISTRO + MODELO_DECLARACION + EJERCICIO + NIF_DECLARANTE
df2 = pd.DataFrame(data=np.array([row_beginning] * count), columns=['Tipo2'], index=range(0,count))

# 18-26 NIF_DECLARADO
# @rosaura Poner dnis invalidos en blanco (' '*9)
# Antes de aplicar 'validateDNI_NIE', eliminar ruido y sanear:
dfaux = pd.DataFrame()
dfaux["tmp"] = df["National Id"].str.replace(r'[\.\-\s\_]+', '', regex=True)
dfaux["tmp"] = dfaux["tmp"].str.upper()
# Validar los DNIs..
dni_mask = dfaux["tmp"].apply(validate_DNI_NIE) 
# ..para poder poner los incorrectos en blanco
dfaux.loc[~dni_mask, ["tmp"]] = ' ' * 9
df2["Tipo2"] += dfaux["tmp"]
df2.loc[2, "Tipo2"]

# 25-37 NIF REPRESENTANTE LEGAL
df2["Tipo2"] += ' ' * 9

# 36-75 APELLIDOS Y NOMBRE 
dfaux["Tipo2"] = df["Family Name"].str.strip()  + ' ' + df["Given Name"].str.strip()
df2["Tipo2"] += ((dfaux["Tipo2"].str.upper()).str.ljust(40).str[:40])


In [65]:
# 76-77 CODIGO DE PROVINCIA
dfprov = pd.read_csv(r".\modelo182\provincias.tsv", sep='\t') # tab sepparated values
# MANY_TO_ONE MERGE
dfaux = df.merge(dfprov, how="left",left_on="Address State", right_on="Provincia")
mask_nan = pd.isna(dfaux["Codigo"])   
dfaux[mask_nan] = 99 # Codigo por defecto para extranjeros
dfaux["Codigo"] = dfaux["Codigo"].astype(int) #merge ha dejado decimales '.0'
df2["Tipo2"]   += (dfaux["Codigo"].astype('str')).str.rjust(2)

# 78 CLAVE
df2["Tipo2"]   += 'A'

### 79-83 PORCENTAJE DE DEDUCCION
![image.png](img\porcentaje_deduccion.png)

In [464]:
# 79-83 PORCENTAJE DE DEDUCCION
# la formula corregida =IF((B4>=C4)AND(C4>=D4)AND(D4>0),1,0) 

#test
df2.to_csv("preview.txt", index=None, header=None)

In [161]:
print('Los nombres de las columnas son:\n \x1b[92m"' + '", "'.join([ col for col in df.columns]) + '"\x1b[0m')
#df["Given Name"] = df["Given Name"].str.upper()


Los nombres de las columnas son:
 "National Id", "Family Name", "Given Name", "Tax State", "Donation Amount", "Currency"
